In [1]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import uniform
import math

In [2]:
# CI function
def CI(data,alpha):
    sample_mean=np.mean(data) # data is a list!
    sample_sigma=np.std(data)
    critical_point = norm.ppf(1-alpha/2)
    LB=sample_mean-critical_point*sample_sigma/np.sqrt(len(data))
    UB=sample_mean+critical_point*sample_sigma/np.sqrt(len(data))
    return LB,UB

# Question 1 :

In [31]:
from scipy.stats import bernoulli
T,N=5,10
r=0.04
S=0.014
size=100000
default_rate=np.array([0.05, 0.01, 0.05, 0.05, 0.01, 0.1, 0.01, 0.09, 0.1, 0.02])
recovery_rate=np.array([0.1, 0.1, 0.3, 0.1, 0.3, 0.1, 0.2, 0.2, 0.1, 0.1])

# Copula matrix
Copula=np.ones((N,N))*0.2
for i in range(N):
    Copula[i,i]=1
#Generate Random variables
mean=np.zeros(N)
payoff=[]
fin_payoff = 1
while abs(fin_payoff) > 0.00001:
    for i in range(size):
        Z=np.random.multivariate_normal(mean, Copula)
        U=norm.cdf(Z)
        default_time=-np.log(1-U)/default_rate

        fifth_indicator=np.where(default_time==sorted(default_time)[4])[0][0]
        tao=default_time[fifth_indicator]
        if tao>T:
            V_prot=S*np.exp(-r)*(1-np.exp(-5*r))/(1-np.exp(-r))
            V_value=0
        else:
            Tj=math.floor(tao)
            V_prot=S*np.exp(-r)*(1-np.exp(-Tj*r))/(1-np.exp(-r))+S*np.exp(-r*tao)*(tao-Tj)
            V_value=(1-recovery_rate[fifth_indicator])*np.exp(-r*tao)
        payoff.append(V_value-V_prot)
    fin_payoff_list = payoff
    fin_payoff = np.mean(fin_payoff_list)
    payoff = []
    S += 0.000001
    if S > 0.015:
        break
else:
    print(fin_payoff, CI(fin_payoff_list, 0.05))

2.6963733705214033e-06 (-0.001299718816496215, 0.0013051115632372577)


In [34]:
np.std(fin_payoff_list)

0.2101364357542546

In [36]:
    for i in range(size):
        Z=np.random.multivariate_normal(mean, Copula)
        U=norm.cdf(Z)
        default_time=-np.log(1-U)/default_rate

        fifth_indicator=np.where(default_time==sorted(default_time)[4])[0][0]
        tao=default_time[fifth_indicator]
        if tao>T:
            V_prot=S*np.exp(-r)*(1-np.exp(-5*r))/(1-np.exp(-r))
            V_value=0
        else:
            Tj=math.floor(tao)
            V_prot=S*np.exp(-r)*(1-np.exp(-Tj*r))/(1-np.exp(-r))+S*np.exp(-r*tao)*(tao-Tj)
            V_value=(1-recovery_rate[fifth_indicator])*np.exp(-r*tao)
        payoff.append(V_value-V_prot)
    fin_payoff_list = payoff
print(fin_payoff, CI(fin_payoff_list, 0.05))

2.6963733705214033e-06 (-0.000620232035561832, 0.001225720762399999)


In [38]:
fin_payoff = np.mean(fin_payoff_list)
print(fin_payoff, CI(fin_payoff_list, 0.05))

0.00030274436341908354 (-0.000620232035561832, 0.001225720762399999)


In [39]:
np.std(fin_payoff_list)

0.21059958101562942

In [30]:
len(fin_payoff_list)

100000

In [4]:
default_time

array([ 5.27679222, 50.76223223, 20.85755165,  9.7899918 , 11.16017817,
        1.04305856, 36.37168819,  2.60643017,  2.30610755, 58.56796248])

In [18]:
np.where(default_time== 0, 1)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [15]:
sorted(default_time)

[1.0430585615601426,
 2.306107551835802,
 2.6064301707817945,
 5.276792218148699,
 9.789991797351718,
 11.160178169456666,
 20.857551653625357,
 36.37168818718845,
 50.76223222795142,
 58.567962479440524]

In [ ]:
Z=np.random.multivariate_normal(mean, Copula)
U=norm.cdf(Z)
default_time=-np.log(1-U)/default_rate


[-444.1700764300307]

In [117]:
np.where(default_time==sorted(default_time)[4])[0][0]

3

0.44

In [119]:
payoff

-444.1700764300307

# Question 2 :

In [55]:
'''
Exit time T follow pdf f(x)=x/18 0<x<6
If T<6, Exit payoff: p*IPO payoff+(1-p)*M&A payoff 
Exit value above 1B would be IPO, all other are M&A
If T=6, exit would be M&A
Series B / M&A payoff = formula 1
Series B / IPO payoff = X*Ib/Pb
Also need to check if XT>1000 or not, 
'''
# parameters
Pa,Pb,Ia,Ib=450,1000,50,100
share_new,share_old,share_current=0.1,0.1,0.8
r,sigma=0.025,0.9
size=1000000

X0=673 # ERROR! Can be adjusted!

U=np.random.uniform(0,1,size)
T=np.sqrt(36*U)

Z=np.random.normal(0,1,size)
X=X0*np.exp((r-0.5*(sigma**2))*T+sigma*np.sqrt(T)*Z)


# Merge = np.max([np.min([Ia*X[np.where(X<1000)]/(Ia+Ib),np.ones(len(np.where(X<1000)[0]))*Ib],axis=0),X[np.where(X<1000)]*Ib/Pb],axis=0)
Merge=np.maximum(np.minimum(Ia*X[X<1000]/(Ia+Ib),np.ones(len(X[X<1000]))*Ib),X[X<1000]*Ib/Pb)
IPO = X[X>=1000]*Ib/Pb
# T 
payoff=np.append(Merge,IPO)*np.exp(-r*T)

print(np.mean(payoff),CI(payoff,0.05))



88.77824550252762 (87.93353067514953, 89.62296032990572)


In [56]:
T

array([2.66496877, 5.7361987 , 2.98468509, ..., 2.76731458, 5.22173241,
       3.52554786])

In [34]:
a=np.array([3,9,15])
b=np.array([4,8,12])
np.maximum(a,b)

array([ 4,  9, 15])

In [24]:
np.min([Ia*X[np.where(X<1000)]/(Ia+Ib),np.ones(len(np.where(X<1000)[0]))*Ib],axis=0)

array([ 40.69501969,  86.6667182 ,  13.52236235, ...,  11.05510907,
        48.32906249, 100.        ])

In [17]:
a=np.min([Ia*X[np.where(X<1000)]/(Ia+Ib),np.ones(len(np.where(X<1000)[0]))*Ib],axis=0)

In [27]:
b=X[np.where(X<1000)]*Ib/Pb
np.max(b)

99.99997516830754

In [18]:
np.max([a,X[np.where(X<1000)]*Ib/Pb],axis=0)

array([1.22470043e+01, 6.35757412e+01, 6.59132382e+01, 5.16888613e+00,
       9.34219766e+00, 8.64093448e+01, 9.31487202e+01, 2.70002260e+00,
       4.55301177e+01, 5.17855663e+01, 8.47032856e+00, 3.52539006e+01,
       7.57262886e+00, 3.05887384e+00, 1.00000000e+02, 4.63605015e+00,
       5.58066935e+00, 1.00000000e+02, 1.08921864e+01, 4.18986828e+00,
       7.80076562e-01, 9.04964622e+01, 3.19720709e+00, 5.07297756e+01,
       1.13387809e+01, 7.39232772e+00, 2.29071169e+01, 9.55450543e+01,
       4.29496873e+01, 4.87277762e+01, 2.84820879e+01, 1.00000000e+02,
       1.00000000e+02, 1.00000000e+02, 1.00000000e+02, 2.52502692e+01,
       4.61025661e+00, 1.00000000e+02, 8.47540270e+01, 1.00000000e+02,
       9.24165825e+01, 1.00000000e+02, 2.42154831e+00, 1.00000000e+02,
       3.87383006e+01, 1.18401777e-01, 4.26935361e+00, 1.00000000e+02,
       9.39142437e+00, 2.71128294e+00, 1.19917934e+01, 1.00000000e+02,
       1.00000000e+02, 3.14210624e+01, 4.99810700e+01, 3.63712927e+01,
      

In [ ]:
# parameters
r,div,sigma,rho=0.05,0.02,0.3,0.2
S10,S20,S30,K,T=100,100,100,100,1
N,size = 252,100000 # N:steps size:simulation times
h=T/N
# simulate standard normal
Z1=np.random.normal(0,1,(N,size))
Z2=np.random.normal(0,1,(N,size))
Z3=np.random.normal(0,1,(N,size))
X1=Z1
X2=rho*Z1+np.sqrt(1-(rho**2))*Z2
X3=rho*Z1+((rho-rho**2)/np.sqrt(1-rho**2))*Z2+(np.sqrt(1-rho**2-((rho-rho**2)**2)/(1-rho**2)))*Z3
# simulate stock price: St=S0*exp{(miu-0.5*sigma^2)*t+sigma*Wt}
S1=np.ones((1,size))*S10
S2=np.ones((1,size))*S20
S3=np.ones((1,size))*S30
zero = np.zeros(size)
for i in tqdm.tqdm(range(N)):
    a=S1[i,:]*np.exp((r-div-(sigma**2)*0.5)*(h)+sigma*np.sqrt(h)*X1[i,:])
    b=S2[i,:]*np.exp((r-div-(sigma**2)*0.5)*(h)+sigma*np.sqrt(h)*X2[i,:])
    c=S3[i,:]*np.exp((r-div-(sigma**2)*0.5)*(h)+sigma*np.sqrt(h)*X3[i,:])
    S1=np.append(S1,[a],axis=0)
    S2=np.append(S2,[b],axis=0)
    S3=np.append(S3,[c],axis=0)
payoff=np.max([S1[-1,:]+S2[-1,:]-S3[-1,:]-K,zero],axis=0) 
# Discount to now